In [1]:


import tensorflow as tf
import matplotlib.pyplot as plt
import numpy
import os


In [2]:

# MNIST 데이터 불러오기
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.mnist.load_data()


In [3]:

X_train = X_train.reshape(X_train.shape[0], 784).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 784).astype('float32')

Y_train = tf.keras.utils.to_categorical(Y_train, 10)
Y_test =  tf.keras.utils.to_categorical(Y_test, 10)




In [4]:

#모델 설계
# method 1
input_Layer = tf.keras.layers.Input(shape=(784,))
x = tf.keras.layers.Dense(512, activation='sigmoid')(input_Layer)
Out_Layer= tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=[input_Layer], outputs=[Out_Layer])
model.summary()


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [5]:

# 모델 컴파일
loss=tf.keras.losses.categorical_crossentropy
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
metric=tf.keras.metrics.categorical_accuracy
model.compile(loss=loss,
             optimizer=optimizer,
             metrics=[metric])

# 베스트 모델 저장을 위한 디렉토리 선언
MODEL_DIR = './MNIST_model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

## 저장한 모델의 주소와 이름.
modelpath="./MNIST_model/{epoch:02d}-{val_loss:.4f}.hdf5"

## 사용할 callback 함수 선언.
callback_list=[tf.keras.callbacks.ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True),
               tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)]

# 모델의 실행
#  validation_data 옵션으로 테스트 데이터만 넣어주어서 검증 데이터 분류가 가능
history = model.fit(X_train, Y_train, validation_split=0.1 , epochs=30, batch_size=200, verbose=1, callbacks=callback_list)



Epoch 1/30
270/270 [==============================] - ETA: 0s - loss: 0.4374 - categorical_accuracy: 0.8806
Epoch 00001: val_loss improved from inf to 0.23525, saving model to ./MNIST_model\01-0.2352.hdf5
270/270 [==============================] - 2s 7ms/step - loss: 0.4374 - categorical_accuracy: 0.8806 - val_loss: 0.2352 - val_categorical_accuracy: 0.9412
Epoch 2/30
264/270 [============================>.] - ETA: 0s - loss: 0.2588 - categorical_accuracy: 0.9276
Epoch 00002: val_loss improved from 0.23525 to 0.19879, saving model to ./MNIST_model\02-0.1988.hdf5
270/270 [==============================] - 2s 6ms/step - loss: 0.2583 - categorical_accuracy: 0.9277 - val_loss: 0.1988 - val_categorical_accuracy: 0.9497
Epoch 3/30
266/270 [============================>.] - ETA: 0s - loss: 0.2241 - categorical_accuracy: 0.9373
Epoch 00003: val_loss improved from 0.19879 to 0.18225, saving model to ./MNIST_model\03-0.1822.hdf5
270/270 [==============================] - 2s 7ms/step - loss: 0.22

KeyboardInterrupt: 

In [ ]:
# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))



In [ ]:
# 테스트 셋의 오차
y_vloss = history.history['val_loss']

# 학습셋의 오차
y_loss = history.history['loss']

# 그래프로 표현
x_len = numpy.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')
# plt.axis([0, 20, 0, 0.35])
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()